In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import cx_Oracle
import config
import time

In [36]:
#!pip install cx_Oracle

#Useless change

In [6]:
'''Conexión a la base de datos'''
#cx_Oracle.init_oracle_client(lib_dir=config.lib_dir)
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding=config.encoding)
    # show the version of the Oracle Database
    print(connection.version)
except cx_Oracle.Error as error:
    print("error:", error)

error: ORA-12170: TNS:Connect timeout occurred


### Consultas Personas Retiradas

In [7]:
# numero de asignaciones facturadas x persona
time_start = datetime.datetime.now()

cur = connection.cursor()
cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_FACTURADAS END AS ASIGNACIONES_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
    SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_FACTURADAS \
    FROM ASIGNACIONES A \
    INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
    INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
    WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
    AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
    AND   AH.FECHA_FIN > AH.FECHA_INICIO \
    AND   A.TARIFA_HORA > 0 \
    GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NOT NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_facturas_persona = pd.DataFrame(res , columns=['PERSONA', 'ASIGNACIONES_FACTURADAS'])
df_facturas_persona.head(2)

AttributeError: 'NoneType' object has no attribute 'cursor'

In [40]:
# numero de asignaciones no facturadas x persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_NO_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_NO_FACTURADAS END AS ASIGNACIONES_NO_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
    SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_NO_FACTURADAS \
    FROM ASIGNACIONES A \
    INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
    INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
    WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
    AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
    AND   AH.FECHA_FIN > AH.FECHA_INICIO \
    AND   A.TARIFA_HORA = 0 \
    GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
 WHERE P.FECHA_RETIRO IS NOT NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_nofacturas_persona = pd.DataFrame(res , columns=['PERSONA', 'ASIGNACIONES_NO_FACTURADAS'])
df_nofacturas_persona.head(2)

Duración de Consulta (seg): 0:00:01.448362


,PERSONA,ASIGNACIONES_NO_FACTURADAS
0,2,0
1,7,6


In [41]:
#Asignaciones de personas por tipos de servicio 
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_TIPOS_SERVICIO IS NULL THEN 0 ELSE B.CANTIDAD_TIPOS_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
            FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(S.TIPO) AS CANTIDAD_TIPOS_SERVICIO \
	FROM ASIGNACIONES A \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO  = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON A.CON_PERSONA  = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON A.COD_SERVICIO = S.CODIGO_SERVICIO \
	WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO,-14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO ,-2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiposervicio_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_TIPOS_SERVICIO'])
pv_tiposervicio_persona = pd.pivot_table(df_tiposervicio_persona, values='CANTIDAD_TIPOS_SERVICIO', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_tiposervicio_persona.columns = [str(col) + '_TP' for col in pv_tiposervicio_persona.columns]
pv_tiposervicio_persona.head(2)

Duración de Consulta (seg): 0:00:24.597621


,AC_TP,ADM_TP,AI_TP,AP_TP,CO_TP,CU_TP,GC_TP,HD_TP,IN_TP,LI_TP,NE_TP,OP_TP,PD_TP,PM_TP,PR_TP,RE_TP,SE_TP,SM_TP,SO_TP,TD_TP
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,11,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0


In [42]:
#Horas de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.HORAS_TIPO_SERVICIO IS NULL THEN 0 ELSE B.HORAS_TIPO_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	S.TIPO                 AS TIPO_SERVICIO, \
	SUM(EA.DURACION_HORAS) AS HORAS_TIPO_SERVICIO \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO        = S.CODIGO_SERVICIO \
	INNER JOIN PERSONAS  P             ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE EA.HORA_FINALIZACION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(P.FECHA_RETIRO , -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_tiposervicio_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','HORAS_TIPO_SERVICIO'])
pv_horas_tiposervicio_persona = pd.pivot_table(df_horas_tiposervicio_persona, values='HORAS_TIPO_SERVICIO', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_horas_tiposervicio_persona.columns = [str(col) + '_HR' for col in pv_horas_tiposervicio_persona.columns]
pv_horas_tiposervicio_persona.head(2)

Duración de Consulta (seg): 0:00:27.669521


,AC_HR,ADM_HR,AI_HR,AP_HR,CO_HR,CU_HR,GC_HR,HD_HR,IN_HR,LI_HR,NE_HR,OP_HR,PD_HR,PM_HR,PR_HR,RE_HR,SE_HR,SM_HR,SO_HR,TD_HR
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0
7,0.0,28.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,2256.0,0


In [43]:
#Tecnologias usada en asignaciones por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.HERRAMIENTA, CASE WHEN B.VECES_HERRAMIENTA_ASIGNADA IS NULL THEN 0 ELSE B.VECES_HERRAMIENTA_ASIGNADA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA,H.NOMBRE AS HERRAMIENTA \
	FROM HERRAMIENTAS H \
	CROSS JOIN PERSONAS P \
    WHERE    P.FECHA_RETIRO IS NOT NULL\
	GROUP BY P.CONSECUTIVO,H.NOMBRE) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO  AS PERSONA,H.NOMBRE  AS HERRAMIENTA,COUNT(AH.CONSECUTIVO) AS VECES_HERRAMIENTA_ASIGNADA \
	FROM HERRAMIENTAS_UTILIZADAS HU \
	INNER JOIN HERRAMIENTAS H             ON HU.CON_HERRAMIENTAS = H.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON HU.COD_SERVICIO     = S.CODIGO_SERVICIO \
	INNER JOIN ASIGNACIONES A             ON S.CODIGO_SERVICIO   = A.COD_SERVICIO \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO       = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON P.CONSECUTIVO       = A.CON_PERSONA \
	WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO, H.NOMBRE) B ON A.PERSONA = B.PERSONA AND  A.HERRAMIENTA = B.HERRAMIENTA \
ORDER BY A.PERSONA, A.HERRAMIENTA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tecno_asig_persona = pd.DataFrame(res , columns=['PERSONA', 'HERRAMIENTA','VECES_HERRAMIENTA_ASIGNADA'])
pv_tecno_asig_persona = pd.pivot_table(df_tecno_asig_persona, values='VECES_HERRAMIENTA_ASIGNADA', index=['PERSONA'], columns='HERRAMIENTA', fill_value=0)
pv_tecno_asig_persona.head(2)

Duración de Consulta (seg): 0:03:29.793139


HERRAMIENTA,.NET,.NET Core,ANGULAR,AWS,Activemq,Alibaba,Android / Java,Android / Kotlin,Angular,Apache Airflow,...,Travis CI,TypeScript,Vue,WEBLOGIC,Websphere,Websphere Message Broker,Wordpress,Xamarin,iOS / Objetive C,iOS / Swift
PERSONA,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
#Horas por etapa por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.ETAPA, CASE WHEN B.HORAS_ETAPA IS NULL THEN 0 ELSE B.HORAS_ETAPA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, E.CODIGO_ETAPA  AS ETAPA \
	FROM ETAPAS  E \
	CROSS JOIN PERSONAS P \
    WHERE  P.FECHA_RETIRO IS NOT NULL\
	GROUP BY P.CONSECUTIVO,E.CODIGO_ETAPA) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	EA.CODIGO_ETAPA        AS ETAPA, \
	SUM(EA.DURACION_HORAS) AS HORAS_ETAPA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN PERSONAS P ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE EA.HORA_FINALIZACION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, EA.CODIGO_ETAPA ) B ON A.PERSONA = B.PERSONA AND A.ETAPA = B.ETAPA \
ORDER BY A.PERSONA, A.ETAPA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_etapa_persona = pd.DataFrame(res , columns=['PERSONA', 'ETAPA','HORAS_ETAPA'])
df_horas_etapa_persona['ETAPA'] = df_horas_etapa_persona['ETAPA'] .fillna('SIN_ETAPA')
pv_horas_etapa_persona = pd.pivot_table(df_horas_etapa_persona, values='HORAS_ETAPA', index=['PERSONA'], columns='ETAPA', fill_value=0)
pv_horas_etapa_persona.head(2)

Duración de Consulta (seg): 0:07:44.244597


ETAPA,ACTCOM,ADMB,ADMBD,ADMCAP,ADMON,ADMSEG,ADMSER,ADMSO,ALEMAN,ANAID,...,TRIBUT,TRINS,TRINSM,TRINT,TRMIG,TRREV,VAC,VAR,VENPRO,VISCLI
PERSONA,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.00,0,0.0,0,0.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
7,0.0,4.0,1881.77,0,0.0,0,0.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0


In [45]:
#Horas en area por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT B.PERSONA, B.AREA, CASE WHEN C.HORAS_AREA IS NULL THEN 0 ELSE C.HORAS_AREA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, A.CONSECUTIVO  AS AREA \
	FROM AREAS  A \
	CROSS JOIN PERSONAS P \
    WHERE P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,A.CONSECUTIVO  ) B \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.CON_AREA  AS AREA, SUM(EA.DURACION_HORAS) AS HORAS_AREA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN PERSONAS P              ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO  = S.CODIGO_SERVICIO \
	WHERE EA.HORA_FINALIZACION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, S.CON_AREA ) C ON B.PERSONA = C.PERSONA AND C.PERSONA =  B.AREA \
ORDER BY B.PERSONA, B.AREA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_area_persona = pd.DataFrame(res , columns=['PERSONA', 'AREA','HORAS_AREA'])
pv_horas_area_persona = pd.pivot_table(df_horas_area_persona, values='HORAS_AREA', index=['PERSONA'], columns='AREA', fill_value=0)
pv_horas_area_persona.head(2)

Duración de Consulta (seg): 0:01:01.745611


AREA,1,2,3,4,5,6,7,8,9,24,...,53,54,55,56,58,59,60,61,62,63
PERSONA,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0.00,0,0.0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
7,0,0,0,0,0,0,571.25,0,0.0,0,...,0,0,0,0.0,0,0.0,0,0,0,0


In [46]:
#Cantidad de pendientes en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_PENDIENTES IS NULL THEN 0 ELSE B.CANTIDAD_PENDIENTES END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(PE.CODIGO_PENDIENTE) AS CANTIDAD_PENDIENTES \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A  ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P  ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN PENDIENTES   PE ON A.COD_SERVICIO = PE.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S  ON PE.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   PE.FECHA_DETECCION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   PE.FECHA_DETECCION < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_pend_tiposerv_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_PENDIENTES'])
pv_pend_tiposerv_persona = pd.pivot_table(df_pend_tiposerv_persona, values='CANTIDAD_PENDIENTES', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_pend_tiposerv_persona.columns = [str(col) + '_PN' for col in pv_pend_tiposerv_persona.columns]
pv_pend_tiposerv_persona.head(2)

Duración de Consulta (seg): 0:00:40.235588


,AC_PN,ADM_PN,AI_PN,AP_PN,CO_PN,CU_PN,GC_PN,HD_PN,IN_PN,LI_PN,NE_PN,OP_PN,PD_PN,PM_PN,PR_PN,RE_PN,SE_PN,SM_PN,SO_PN,TD_PN
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,261,0,0,0,0,0,0,0,0,0,0,0,29,0,0,0,0,0,0


In [47]:
#Cantidad de riesgos en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_RIESGOS IS NULL THEN 0 ELSE B.CANTIDAD_RIESGOS END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(RS.CONSECUTIVO) AS CANTIDAD_RIESGOS \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A      ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P      ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN RIESGOS_SERVICIO RS ON A.COD_SERVICIO = RS.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S      ON RS.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE AH.FECHA_FIN BETWEEN ADD_MONTHS(P.FECHA_RETIRO, -14) AND ADD_MONTHS(P.FECHA_RETIRO, -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   RS.FECHA_REGISTRO BETWEEN ADD_MONTHS(P.FECHA_RETIRO, -14)  AND ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA, A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_ries_tiposerv_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_RIESGOS'])
pv_ries_tiposerv_persona = pd.pivot_table(df_ries_tiposerv_persona, values='CANTIDAD_RIESGOS', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_ries_tiposerv_persona.columns = [str(col) + '_RS' for col in pv_ries_tiposerv_persona.columns]
pv_ries_tiposerv_persona.head(2)

Duración de Consulta (seg): 0:00:30.931114


,AC_RS,ADM_RS,AI_RS,AP_RS,CO_RS,CU_RS,GC_RS,HD_RS,IN_RS,LI_RS,NE_RS,OP_RS,PD_RS,PM_RS,PR_RS,RE_RS,SE_RS,SM_RS,SO_RS,TD_RS
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
#Edad personas desde su fecha de nacimiento hasta el momento de retirarse
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, FECHA_NACIMIENTO, FECHA_RETIRO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NOT NULL AND FECHA_NACIMIENTO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_edad_persona = pd.DataFrame(res , columns=['PERSONA', 'FECHA_NACIMIENTO', 'FECHA_RETIRO'])

#Function to calculate people's age
def age(date):
    birth_date = date[0]
    retire_date = date[1]
    age = retire_date.year - birth_date.year - ((retire_date.month, retire_date.day) < (birth_date.month, birth_date.day))
    return age

df_edad_persona['EDAD'] = df_edad_persona[['FECHA_NACIMIENTO', 'FECHA_RETIRO']].apply(age, axis = 1)
df_edad_persona.drop(columns = ['FECHA_NACIMIENTO', 'FECHA_RETIRO'], inplace = True)
df_edad_persona.head(2)

Duración de Consulta (seg): 0:00:01.133984


,PERSONA,EDAD
0,553,43
1,560,35


In [45]:
df_edad_persona['FECHA_NACIMIENTO'].isna().value_counts()

False    1121
True      257
Name: FECHA_NACIMIENTO, dtype: int64

In [49]:
#Género de la persona: 0 para M y 1 para F
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, GENERO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NOT NULL AND FECHA_NACIMIENTO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_genero_persona = pd.DataFrame(res , columns=['PERSONA', 'GENERO'])
df_genero_persona.replace(to_replace = ['M', 'F'], value = [0, 1], inplace = True)
df_genero_persona.head(2)

Duración de Consulta (seg): 0:00:01.107130


,PERSONA,GENERO
0,553,0
1,560,0


In [50]:
#Tiempo que la persona ha estado en la compañía
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, FECHA_INGRESO, FECHA_RETIRO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NOT NULL AND FECHA_INGRESO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiempo_emp_persona = pd.DataFrame(res , columns=['PERSONA', 'FECHA_INGRESO', 'FECHA_RETIRO'])

#Function to calculate people time in the company
def time(date):
    enter_date = date[0]
    retire_date = date[1]
    months = (retire_date.year - enter_date.year)*12 + (retire_date.month - enter_date.month) 
    return months

df_tiempo_emp_persona['TIEMPO_EMP'] = df_tiempo_emp_persona[['FECHA_INGRESO', 'FECHA_RETIRO']].apply(time, axis = 1)
df_tiempo_emp_persona.drop(columns = ['FECHA_INGRESO', 'FECHA_RETIRO'], inplace = True)
df_tiempo_emp_persona.head(2)

Duración de Consulta (seg): 0:00:01.267667


,PERSONA,TIEMPO_EMP
0,553,24
1,560,33


### Creación Dataset Retirados

In [51]:
time_start = datetime.datetime.now()

print('asignaciones facturadas x persona:',df_facturas_persona.shape)
print('asignaciones no facturadas x persona:',df_nofacturas_persona.shape)
print('numero tipos servicio x persona:',pv_tiposervicio_persona.shape)
print('horas tipo servicio x persona:',pv_horas_tiposervicio_persona.shape)
print('tecnologias usadas x persona:',pv_tecno_asig_persona.shape)
print('horas en etapa x persona:',pv_horas_etapa_persona.shape)
print('horas en area x persona:',pv_horas_area_persona.shape)
print('pendientes en tipos de servicio x persona:',pv_pend_tiposerv_persona.shape)
print('riesgos en tipos de servicio x persona:',pv_ries_tiposerv_persona.shape)
print('edad en años x persona al momento de retirarse:',df_edad_persona.shape)
print('genero x persona:',df_genero_persona.shape)
print('tiempo en meses en la compañía x persona:',df_tiempo_emp_persona.shape)

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

asignaciones facturadas x persona: (1383, 2)
asignaciones no facturadas x persona: (1383, 2)
numero tipos servicio x persona: (1383, 20)
horas tipo servicio x persona: (1383, 20)
tecnologias usadas x persona: (1383, 169)
horas en etapa x persona: (1383, 374)
horas en area x persona: (1383, 44)
pendientes en tipos de servicio x persona: (1383, 20)
riesgos en tipos de servicio x persona: (1383, 20)
edad en años x persona al momento de retirarse: (1126, 2)
genero x persona: (1126, 2)
tiempo en meses en la compañía x persona: (1383, 2)
Duración de Consulta (seg): 0:00:00.002994


In [52]:
time_start = datetime.datetime.now()

temp_dataset_1 = pd.merge(pv_tiposervicio_persona,
                          pd.merge(df_facturas_persona,df_nofacturas_persona, left_on='PERSONA', right_on='PERSONA'),left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_horas_tiposervicio_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_tecno_asig_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_horas_etapa_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_horas_area_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_pend_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_ries_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,df_edad_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,df_genero_persona, left_on='PERSONA', right_on='PERSONA')
df_dataset_retirados = pd.merge(temp_dataset_1,df_tiempo_emp_persona, left_on='PERSONA', right_on='PERSONA')
vc_dataset_retirados = df_dataset_retirados.values.tolist()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

print(len(df_dataset_retirados))
df_dataset_retirados.head()

Duración de Consulta (seg): 0:00:00.385361
1126


,PERSONA,AC_TP,ADM_TP,AI_TP,AP_TP,CO_TP,CU_TP,GC_TP,HD_TP,IN_TP,...,PM_RS,PR_RS,RE_RS,SE_RS,SM_RS,SO_RS,TD_RS,EDAD,GENERO,TIEMPO_EMP
0,7,0,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,46,0,183
1,9,1,57,0,4,5,0,0,0,0,...,0,0,0,0,0,0,0,44,1,138
2,12,0,23,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,40,0,177
3,28,0,28,0,9,6,0,0,0,0,...,0,0,0,0,95,0,0,38,0,116
4,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,37,0,168


In [53]:
#Exporting the datsets to csv
df_dataset_retirados.to_csv('retired_people.csv', index = False)

### Consultas Personas Vigentes

In [54]:
# numero de asignaciones facturadas x persona
time_start = datetime.datetime.now()

cur = connection.cursor()
cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_FACTURADAS END AS ASIGNACIONES_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_FACTURADAS \
FROM ASIGNACIONES A \
INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
WHERE AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
AND   AH.FECHA_FIN > AH.FECHA_INICIO \
AND   A.TARIFA_HORA > 0 \
GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_facturas_persona = pd.DataFrame(res , columns=['PERSONA', 'ASIGNACIONES_FACTURADAS'])
df_facturas_persona.head(2)

Duración de Consulta (seg): 0:00:00.749705


,PERSONA,ASIGNACIONES_FACTURADAS
0,6,0
1,8,0


In [55]:
# numero de asignaciones no facturadas x persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_NO_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_NO_FACTURADAS END AS ASIGNACIONES_NO_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_NO_FACTURADAS \
FROM ASIGNACIONES A \
INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
WHERE AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
AND   AH.FECHA_FIN > AH.FECHA_INICIO \
AND   A.TARIFA_HORA = 0 \
GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_nofacturas_persona = pd.DataFrame(res , columns=['PERSONA', 'ASIGNACIONES_NO_FACTURADAS'])
df_nofacturas_persona.head(2)

Duración de Consulta (seg): 0:00:00.746381


,PERSONA,ASIGNACIONES_NO_FACTURADAS
0,6,15
1,8,12


In [56]:
#Asignaciones de personas por tipos de servicio 
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_TIPOS_SERVICIO IS NULL THEN 0 ELSE B.CANTIDAD_TIPOS_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(S.TIPO) AS CANTIDAD_TIPOS_SERVICIO \
	FROM ASIGNACIONES A \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO  = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON A.CON_PERSONA  = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON A.COD_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiposervicio_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_TIPOS_SERVICIO'])
pv_tiposervicio_persona = pd.pivot_table(df_tiposervicio_persona, values='CANTIDAD_TIPOS_SERVICIO', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_tiposervicio_persona.columns = [str(col) + '_TP' for col in pv_tiposervicio_persona.columns]
pv_tiposervicio_persona.head(2)

Duración de Consulta (seg): 0:00:11.413366


,AC_TP,ADM_TP,AI_TP,AP_TP,CO_TP,CU_TP,GC_TP,HD_TP,IN_TP,LI_TP,NE_TP,OP_TP,PD_TP,PM_TP,PR_TP,RE_TP,SE_TP,SM_TP,SO_TP,TD_TP
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,1,16,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0


In [57]:
#Horas de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.HORAS_TIPO_SERVICIO IS NULL THEN 0 ELSE B.HORAS_TIPO_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	S.TIPO                 AS TIPO_SERVICIO, \
	SUM(EA.DURACION_HORAS) AS HORAS_TIPO_SERVICIO \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO        = S.CODIGO_SERVICIO \
	INNER JOIN PERSONAS  P             ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE P.ESTADO = 1 \
	AND EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE) , -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_tiposervicio_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','HORAS_TIPO_SERVICIO'])
pv_horas_tiposervicio_persona = pd.pivot_table(df_horas_tiposervicio_persona, values='HORAS_TIPO_SERVICIO', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_horas_tiposervicio_persona.columns = [str(col) + '_HR' for col in pv_horas_tiposervicio_persona.columns]
pv_horas_tiposervicio_persona.head(2)

Duración de Consulta (seg): 0:00:16.241337


,AC_HR,ADM_HR,AI_HR,AP_HR,CO_HR,CU_HR,GC_HR,HD_HR,IN_HR,LI_HR,NE_HR,OP_HR,PD_HR,PM_HR,PR_HR,RE_HR,SE_HR,SM_HR,SO_HR,TD_HR
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0.0,846.00,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
8,0.0,986.24,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0


In [58]:
#Tecnologias usada en asignaciones por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.HERRAMIENTA, CASE WHEN B.VECES_HERRAMIENTA_ASIGNADA IS NULL THEN 0 ELSE B.VECES_HERRAMIENTA_ASIGNADA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA,H.NOMBRE AS HERRAMIENTA \
	FROM HERRAMIENTAS H	 \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,H.NOMBRE) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO  AS PERSONA,H.NOMBRE  AS HERRAMIENTA,COUNT(AH.CONSECUTIVO) AS VECES_HERRAMIENTA_ASIGNADA \
	FROM HERRAMIENTAS_UTILIZADAS HU \
	INNER JOIN HERRAMIENTAS H             ON HU.CON_HERRAMIENTAS = H.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON HU.COD_SERVICIO     = S.CODIGO_SERVICIO \
	INNER JOIN ASIGNACIONES A             ON S.CODIGO_SERVICIO   = A.COD_SERVICIO \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO       = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON P.CONSECUTIVO       = A.CON_PERSONA \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, H.NOMBRE) B ON A.PERSONA = B.PERSONA AND  A.HERRAMIENTA = B.HERRAMIENTA \
ORDER BY A.PERSONA, A.HERRAMIENTA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tecno_asig_persona = pd.DataFrame(res , columns=['PERSONA', 'HERRAMIENTA','VECES_HERRAMIENTA_ASIGNADA'])
pv_tecno_asig_persona = pd.pivot_table(df_tecno_asig_persona, values='VECES_HERRAMIENTA_ASIGNADA', index=['PERSONA'], columns='HERRAMIENTA', fill_value=0)
pv_tecno_asig_persona.head(2)

Duración de Consulta (seg): 0:01:29.544484


HERRAMIENTA,.NET,.NET Core,ANGULAR,AWS,Activemq,Alibaba,Android / Java,Android / Kotlin,Angular,Apache Airflow,...,Travis CI,TypeScript,Vue,WEBLOGIC,Websphere,Websphere Message Broker,Wordpress,Xamarin,iOS / Objetive C,iOS / Swift
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
#Horas por etapa por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.ETAPA, CASE WHEN B.HORAS_ETAPA IS NULL THEN 0 ELSE B.HORAS_ETAPA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, E.CODIGO_ETAPA  AS ETAPA \
	FROM ETAPAS  E \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
    AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, E.CODIGO_ETAPA) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	EA.CODIGO_ETAPA        AS ETAPA, \
	SUM(EA.DURACION_HORAS) AS HORAS_ETAPA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN PERSONAS P ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE P.ESTADO = 1 \
	AND   EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, EA.CODIGO_ETAPA ) B ON A.PERSONA = B.PERSONA AND A.ETAPA = B.ETAPA \
ORDER BY A.PERSONA, A.ETAPA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_etapa_persona = pd.DataFrame(res , columns=['PERSONA', 'ETAPA','HORAS_ETAPA'])
df_horas_etapa_persona['ETAPA'] = df_horas_etapa_persona['ETAPA'] .fillna('SIN_ETAPA')
pv_horas_etapa_persona = pd.pivot_table(df_horas_etapa_persona, values='HORAS_ETAPA', index=['PERSONA'], columns='ETAPA', fill_value=0)
pv_horas_etapa_persona.head(2)

Duración de Consulta (seg): 0:03:15.819167


ETAPA,ACTCOM,ADMB,ADMBD,ADMCAP,ADMON,ADMSEG,ADMSER,ADMSO,ALEMAN,ANAID,...,TRIBUT,TRINS,TRINSM,TRINT,TRMIG,TRREV,VAC,VAR,VENPRO,VISCLI
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0.0,0,0.0,0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,56.0,0,0.0,0.0
8,0,0,0.0,0,0.0,0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,120.0,0,0.0,0.0


In [60]:
#Horas en area por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT B.PERSONA, B.AREA, CASE WHEN C.HORAS_AREA IS NULL THEN 0 ELSE C.HORAS_AREA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, A.CONSECUTIVO  AS AREA \
	FROM AREAS  A \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
    AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,A.CONSECUTIVO  ) B \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.CON_AREA  AS AREA, SUM(EA.DURACION_HORAS) AS HORAS_AREA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN PERSONAS P              ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO  = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, S.CON_AREA ) C ON B.PERSONA = C.PERSONA AND C.PERSONA =  B.AREA \
ORDER BY B.PERSONA, B.AREA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_area_persona = pd.DataFrame(res , columns=['PERSONA', 'AREA','HORAS_AREA'])
pv_horas_area_persona = pd.pivot_table(df_horas_area_persona, values='HORAS_AREA', index=['PERSONA'], columns='AREA', fill_value=0)
pv_horas_area_persona.head(2)

Duración de Consulta (seg): 0:00:28.447817


AREA,1,2,3,4,5,6,7,8,9,24,...,53,54,55,56,58,59,60,61,62,63
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,120.857143,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0.000000,0,164.373333,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#Cantidad de pendientes en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_PENDIENTES IS NULL THEN 0 ELSE B.CANTIDAD_PENDIENTES END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND    C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(PE.CODIGO_PENDIENTE) AS CANTIDAD_PENDIENTES \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A  ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P  ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN PENDIENTES   PE ON A.COD_SERVICIO = PE.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S  ON PE.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   PE.FECHA_DETECCION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   PE.FECHA_DETECCION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_pend_tiposerv_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_PENDIENTES'])
pv_pend_tiposerv_persona = pd.pivot_table(df_pend_tiposerv_persona, values='CANTIDAD_PENDIENTES', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_pend_tiposerv_persona.columns = [str(col) + '_PN' for col in pv_pend_tiposerv_persona.columns]
pv_pend_tiposerv_persona.head(2)

Duración de Consulta (seg): 0:00:15.987945


,AC_PN,ADM_PN,AI_PN,AP_PN,CO_PN,CU_PN,GC_PN,HD_PN,IN_PN,LI_PN,NE_PN,OP_PN,PD_PN,PM_PN,PR_PN,RE_PN,SE_PN,SM_PN,SO_PN,TD_PN
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,2070,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,0,547,0,0,0,0,0,0,0,0,0,0,0,18,0,0,0,0,0,0


In [62]:
#Cantidad de riesgos en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_RIESGOS IS NULL THEN 0 ELSE B.CANTIDAD_RIESGOS END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND    C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(RS.CONSECUTIVO) AS CANTIDAD_RIESGOS \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A      ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P      ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN RIESGOS_SERVICIO RS ON A.COD_SERVICIO = RS.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S      ON RS.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN BETWEEN ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) AND ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   RS.FECHA_REGISTRO BETWEEN ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14)  AND ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA, A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_ries_tiposerv_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_RIESGOS'])
pv_ries_tiposerv_persona = pd.pivot_table(df_ries_tiposerv_persona, values='CANTIDAD_RIESGOS', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_ries_tiposerv_persona.columns = [str(col) + '_RS' for col in pv_ries_tiposerv_persona.columns]
pv_ries_tiposerv_persona.head(2)

Duración de Consulta (seg): 0:00:11.183774


,AC_RS,ADM_RS,AI_RS,AP_RS,CO_RS,CU_RS,GC_RS,HD_RS,IN_RS,LI_RS,NE_RS,OP_RS,PD_RS,PM_RS,PR_RS,RE_RS,SE_RS,SM_RS,SO_RS,TD_RS
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
#Edad personas desde su fecha de nacimiento hasta hoy
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, FECHA_NACIMIENTO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NULL AND FECHA_NACIMIENTO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_edad_persona = pd.DataFrame(res , columns=['PERSONA', 'FECHA_NACIMIENTO'])

#Function to calculate people's age
def age(birth_date):
    current_date = datetime.datetime(2021, 12, 1) #Y-M-D
    age = current_date.year - birth_date.year - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    return age

df_edad_persona['EDAD'] = df_edad_persona['FECHA_NACIMIENTO'].apply(age)
df_edad_persona.drop(columns = ['FECHA_NACIMIENTO'], inplace = True)
df_edad_persona.head(2)

Duración de Consulta (seg): 0:00:00.617424


,PERSONA,EDAD
0,559,52
1,565,50


In [64]:
#Género de la persona: 0 para M y 1 para F
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, GENERO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NULL AND FECHA_NACIMIENTO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_genero_persona = pd.DataFrame(res , columns=['PERSONA', 'GENERO'])
df_genero_persona.replace(to_replace = ['M', 'F'], value = [0, 1], inplace = True)
df_genero_persona.head(2)

Duración de Consulta (seg): 0:00:00.600545


,PERSONA,GENERO
0,559,0
1,565,1


In [65]:
#Tiempo que la persona ha estado en la compañía
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, FECHA_INGRESO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NULL AND FECHA_INGRESO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiempo_emp_persona = pd.DataFrame(res , columns=['PERSONA', 'FECHA_INGRESO'])

#Function to calculate people time in the company
def time(date):
    enter_date = date
    current_date = datetime.datetime(2021, 12, 1) #Y-M-D
    months = (current_date.year - enter_date.year)*12 + (current_date.month - enter_date.month) 
    return months

df_tiempo_emp_persona['TIEMPO_EMP'] = df_tiempo_emp_persona['FECHA_INGRESO'].apply(time)
df_tiempo_emp_persona.drop(columns = ['FECHA_INGRESO'], inplace = True)
df_tiempo_emp_persona.head(2)

Duración de Consulta (seg): 0:00:00.598914


,PERSONA,TIEMPO_EMP
0,559,118
1,565,118


In [66]:
time_start = datetime.datetime.now()

print('asignaciones facturadas x persona:',df_facturas_persona.shape)
print('asignaciones no facturadas x persona:',df_nofacturas_persona.shape)
print('numero tipos servicio x persona:',pv_tiposervicio_persona.shape)
print('horas tipo servicio x persona:',pv_horas_tiposervicio_persona.shape)
print('tecnologias usadas x persona:',pv_tecno_asig_persona.shape)
print('horas en etapa x persona:',pv_horas_etapa_persona.shape)
print('horas en area x persona:',pv_horas_area_persona.shape)
print('pendientes en tipos de servicio x persona:',pv_pend_tiposerv_persona.shape)
print('riesgos en tipos de servicio x persona:',pv_ries_tiposerv_persona.shape)
print('edad en años x persona al momento de retirarse:',df_edad_persona.shape)
print('genero x persona:',df_genero_persona.shape)
print('tiempo en meses en la compañía x persona:',df_tiempo_emp_persona.shape)

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

asignaciones facturadas x persona: (591, 2)
asignaciones no facturadas x persona: (591, 2)
numero tipos servicio x persona: (588, 20)
horas tipo servicio x persona: (588, 20)
tecnologias usadas x persona: (588, 169)
horas en etapa x persona: (588, 374)
horas en area x persona: (588, 44)
pendientes en tipos de servicio x persona: (588, 20)
riesgos en tipos de servicio x persona: (588, 20)
edad en años x persona al momento de retirarse: (590, 2)
genero x persona: (590, 2)
tiempo en meses en la compañía x persona: (590, 2)
Duración de Consulta (seg): 0:00:00.000963


In [67]:
time_start = datetime.datetime.now()

temp_dataset_1 = pd.merge(pv_tiposervicio_persona,
                          pd.merge(df_facturas_persona,df_nofacturas_persona, left_on='PERSONA', right_on='PERSONA'),left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_horas_tiposervicio_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_tecno_asig_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_horas_etapa_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_horas_area_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_pend_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_ries_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,df_edad_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,df_genero_persona, left_on='PERSONA', right_on='PERSONA')
df_dataset_vigentes = pd.merge(temp_dataset_1,df_tiempo_emp_persona, left_on='PERSONA', right_on='PERSONA')
vc_dataset_vigentes = df_dataset_vigentes.values.tolist()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

print(len(df_dataset_vigentes))
df_dataset_vigentes.head()

Duración de Consulta (seg): 0:00:00.215467
588


,PERSONA,AC_TP,ADM_TP,AI_TP,AP_TP,CO_TP,CU_TP,GC_TP,HD_TP,IN_TP,...,PM_RS,PR_RS,RE_RS,SE_RS,SM_RS,SO_RS,TD_RS,EDAD,GENERO,TIEMPO_EMP
0,6,0,27,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,59,0,366
1,8,1,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,41,1,184
2,21,0,29,1,9,0,0,0,0,0,...,0,0,0,0,224,0,0,41,0,194
3,31,0,22,0,6,0,0,0,0,0,...,0,0,0,0,152,0,0,46,0,149
4,35,0,20,0,2,0,0,0,0,1,...,0,0,0,0,62,0,0,58,0,366


In [68]:
#Exporting the dataset to csv
df_dataset_vigentes.to_csv('non_retired_people.csv', index = False)

In [39]:
if connection:
    connection.close()

In [ ]:
df_dataset_vigentes
vc_dataset_vigentes
df_dataset_retirados
vc_dataset_retirados